In [1]:
# =============================================================================
# CELL 1: Setup and Directory Structure
# =============================================================================

import sys
from pathlib import Path

# Setup paths
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Configuration helper
def create_config(overrides=None):
    from config import PipelineConfig
    config = PipelineConfig(overrides)
    config.project_root = project_root
    config.documents_root = project_root / "documents"
    config.literature_folder = config.documents_root / "literature"
    config.your_work_folder = config.documents_root / "your_work"
    config.biblio_folder = config.documents_root / "biblio"
    config.current_drafts_folder = config.documents_root / "current_drafts"
    config.cache_file = project_root / "physics_knowledge_base.pkl"
    config._create_directories()
    return config

# Create configuration and directories
config = create_config()

print("📁 YOUR PHYSICS PIPELINE DIRECTORY STRUCTURE")
print("=" * 60)
print(f"Project Root: {project_root}")
print()

# Show where to put files
folders = {
    "biblio": "Put your Zotero .bib files here",
    "literature": "Downloaded papers go here (auto-created)",
    "your_work": "Your own published papers",
    "current_drafts": "Your current drafts"
}

for folder_name, description in folders.items():
    folder_path = config.documents_root / folder_name
    exists = "✅" if folder_path.exists() else "❌"
    file_count = len(list(folder_path.iterdir())) if folder_path.exists() else 0
    
    print(f"{exists} {folder_name}/ - {description}")
    print(f"   📍 Location: {folder_path}")
    print(f"   📊 Current files: {file_count}")
    print()

print("🎯 TO GET STARTED:")
print("1. Export .bib files from Zotero → Save to documents/biblio/")
print("2. Copy your papers → Save to documents/your_work/")
print("3. Run the commands below!")

📁 YOUR PHYSICS PIPELINE DIRECTORY STRUCTURE
Project Root: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular

✅ biblio/ - Put your Zotero .bib files here
   📍 Location: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/biblio
   📊 Current files: 2

✅ literature/ - Downloaded papers go here (auto-created)
   📍 Location: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/literature
   📊 Current files: 1

✅ your_work/ - Your own published papers
   📍 Location: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/your_work
   📊 Current files: 10

✅ current_drafts/ - Your current drafts
   📍 Location: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/current_drafts
   📊 Current files: 2

🎯 TO GET STARTED:
1. Export .bib files from Zotero → Save 

In [2]:
# =============================================================================
# CELL 2: Check for Existing .bib Files
# =============================================================================

# Check what .bib files you already have
biblio_folder = config.biblio_folder
bib_files = list(biblio_folder.glob("*.bib"))

print("📚 CHECKING FOR EXISTING .BIB FILES")
print("=" * 40)

if bib_files:
    print(f"✅ Found {len(bib_files)} .bib file(s):")
    for bib_file in bib_files:
        file_size = bib_file.stat().st_size / 1024  # KB
        print(f"   📄 {bib_file.name} ({file_size:.1f} KB)")
    
    # Quick preview of first .bib file
    first_bib = bib_files[0]
    try:
        content = first_bib.read_text()
        lines = content.split('\n')[:10]
        print(f"\n📖 Preview of {first_bib.name}:")
        for line in lines:
            if line.strip():
                print(f"   {line[:60]}...")
                break
    except Exception as e:
        print(f"   Could not preview file: {e}")
        
    print(f"\n🚀 Ready to download literature!")
    
else:
    print("📭 No .bib files found")
    print("\n💡 TO ADD .BIB FILES:")
    print("1. Open Zotero")
    print("2. Select your collection")  
    print("3. Right-click → Export Collection")
    print("4. Choose 'BibTeX' format")
    print(f"5. Save to: {biblio_folder}")

📚 CHECKING FOR EXISTING .BIB FILES
✅ Found 1 .bib file(s):
   📄 DynamicsConditionalEnsembles.bib (367.3 KB)

📖 Preview of DynamicsConditionalEnsembles.bib:
   @article{A.S.Holevo1998CapacityQuantumChannel,...

🚀 Ready to download literature!


In [3]:
# =============================================================================
# CELL 3: Download Literature from .bib Files (IF YOU HAVE THEM)
# =============================================================================

# Only run this if you have .bib files
bib_files = list(config.biblio_folder.glob("*.bib"))

if bib_files:
    print("🚀 DOWNLOADING LITERATURE FROM ARXIV")
    print("=" * 45)
    
    from src.downloaders import LiteratureDownloader
    
    # Use the first .bib file (or modify to use a specific one)
    bib_file = bib_files[0]
    print(f"📖 Using: {bib_file.name}")
    
    # Initialize downloader
    downloader = LiteratureDownloader(
        output_directory=config.literature_folder,
        delay_between_downloads=config.download_delay,
        arxiv_config=config.get_arxiv_config()
    )
    
    print("⏳ Starting download... (this may take several minutes)")
    print("💡 The system will:")
    print("   1. Parse your .bib file")
    print("   2. Search for papers on arXiv")
    print("   3. Download PDFs and TEX files")
    print("   4. Generate a detailed report")
    print()
    
    # Download papers
    results = downloader.download_from_bibtex(bib_file)
    
    print(f"\n✅ DOWNLOAD COMPLETE!")
    print(f"   📥 Successfully downloaded: {len(results['successful'])}")
    print(f"   ❌ Not found: {len(results['failed'])}")
    
    if results['successful']:
        print(f"\n📄 Sample downloads:")
        for i, result in enumerate(results['successful'][:3]):
            paper = result.paper_metadata
            print(f"   {i+1}. {paper.title[:50]}...")
            print(f"      arXiv: {result.search_result.arxiv_id}")
            
else:
    print("⏭️ SKIPPING DOWNLOAD - No .bib files found")
    print("Add .bib files to documents/biblio/ first!")

🚀 DOWNLOADING LITERATURE FROM ARXIV
📖 Using: DynamicsConditionalEnsembles.bib
2025-06-02 08:44:54 - physics_pipeline.src.downloaders.bibtex_parser - INFO - BibTeX parser initialized
2025-06-02 08:44:54 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - ArXiv searcher initialized
2025-06-02 08:44:54 - physics_pipeline.src.downloaders.literature_downloader - INFO - Literature downloader initialized with output: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/literature
⏳ Starting download... (this may take several minutes)
💡 The system will:
   1. Parse your .bib file
   2. Search for papers on arXiv
   3. Download PDFs and TEX files
   4. Generate a detailed report

2025-06-02 08:44:54 - physics_pipeline.src.downloaders.literature_downloader - INFO - Starting download process from /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/biblio/DynamicsConditionalEnsembles.bib
202

2025-06-02 08:45:26 - physics_pipeline.src.utils.file_utils - INFO - Extracted main tex file: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/literature/1808.05949.tex
2025-06-02 08:45:26 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - TEX downloaded and extracted: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/literature/1808.05949.tex
2025-06-02 08:45:27 - physics_pipeline.src.downloaders.literature_downloader - INFO - ✓ Successfully downloaded: 1808.05949
2025-06-02 08:45:27 - physics_pipeline.src.downloaders.literature_downloader - INFO - Processing paper 8/256: Many-Body Quantum Chaos and Space-time Translation...
2025-06-02 08:45:27 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Searching for paper: Many-Body Quantum Chaos and Space-time Translation...
2025-06-02 08:45:29 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Downloading paper:

2025-06-02 08:46:09 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - TEX downloaded and extracted: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/literature/2008.08683.tex
2025-06-02 08:46:10 - physics_pipeline.src.downloaders.literature_downloader - INFO - ✓ Successfully downloaded: 2008.08683
2025-06-02 08:46:10 - physics_pipeline.src.downloaders.literature_downloader - INFO - Processing paper 17/256: Quantum Information Dimension and Geometric Entrop...
2025-06-02 08:46:10 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Searching for paper: Quantum Information Dimension and Geometric Entrop...
2025-06-02 08:46:12 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Downloading paper: 2111.06374
2025-06-02 08:46:41 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - PDF downloaded: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/literature/2

2025-06-02 08:47:56 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - TEX downloaded and extracted: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/literature/1901.06688.tex
2025-06-02 08:47:57 - physics_pipeline.src.downloaders.literature_downloader - INFO - ✓ Successfully downloaded: 1901.06688
2025-06-02 08:47:57 - physics_pipeline.src.downloaders.literature_downloader - INFO - Processing paper 26/256: Stochastic Differential Equations and a Posteriori...
2025-06-02 08:47:57 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Searching for paper: Stochastic Differential Equations and a Posteriori...
2025-06-02 08:48:02 - physics_pipeline.src.downloaders.literature_downloader - WARNING - ✗ Failed to download: Stochastic Differential Equations and a Posteriori...
2025-06-02 08:48:02 - physics_pipeline.src.downloaders.literature_downloader - INFO - Processing paper 27/256: No-Faster-than-Light-Signaling Implies Line

2025-06-02 08:48:40 - physics_pipeline.src.downloaders.literature_downloader - WARNING - ✗ Failed to download: Geometrization of Quantum Mechanics...
2025-06-02 08:48:40 - physics_pipeline.src.downloaders.literature_downloader - INFO - Processing paper 35/256: Quantum Steering: A Review with Focus on Semidefin...
2025-06-02 08:48:40 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Searching for paper: Quantum Steering: A Review with Focus on Semidefin...
2025-06-02 08:48:41 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Downloading paper: 1604.00501
2025-06-02 08:48:42 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - PDF downloaded: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/literature/1604.00501.pdf
2025-06-02 08:48:42 - physics_pipeline.src.utils.file_utils - INFO - Extracted main tex file: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/l

...

2025-06-02 09:03:32 - physics_pipeline.src.downloaders.literature_downloader - INFO - ✓ Successfully downloaded: 2107.11354
2025-06-02 09:03:32 - physics_pipeline.src.downloaders.literature_downloader - INFO - Processing paper 222/256: Diagnostics of Entanglement Dynamics in Noisy and ...
2025-06-02 09:03:32 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Searching for paper: Diagnostics of Entanglement Dynamics in Noisy and ...
2025-06-02 09:03:35 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Downloading paper: 2107.11354
2025-06-02 09:03:36 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - PDF downloaded: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/literature/2107.11354.pdf
2025-06-02 09:03:36 - physics_pipeline.src.utils.file_utils - INFO - Extracted main tex file: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/literature/2107.11354.tex


2025-06-02 09:04:03 - physics_pipeline.src.downloaders.literature_downloader - INFO - Processing paper 229/256: Full Counting Statistics across the Entanglement P...
2025-06-02 09:04:03 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Searching for paper: Full Counting Statistics across the Entanglement P...
2025-06-02 09:04:04 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Downloading paper: 2302.09470
2025-06-02 09:04:05 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - PDF downloaded: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/literature/2302.09470.pdf
2025-06-02 09:04:06 - physics_pipeline.src.utils.file_utils - INFO - Extracted main tex file: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/literature/2302.09470.tex
2025-06-02 09:04:06 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - TEX downloaded and extracted: /Users/fanza/De

2025-06-02 09:04:36 - physics_pipeline.src.downloaders.arxiv_searcher - ERROR - Error in Google search: 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://www.google.com/search%3Fq%3D%2522Smoothness%2Bof%2BWave%2BFunctions%2Bin%2BThermal%2BEquilibrium%2522%2Bsite%253Aarxiv.org%26num%3D7%26hl%3Den%26start%3D0%26safe%3Dactive&hl=en&q=EgSCVfKaGOTC9sEGIjA9xxkKTlByMfIspAAP1XBfEy108o9MYChaJZOaJe1LzboTjwsVJWBkkjyRTzMO2MAyAnJSWgFD
2025-06-02 09:04:36 - physics_pipeline.src.downloaders.literature_downloader - WARNING - ✗ Failed to download: Smoothness of Wave Functions in Thermal Equilibriu...
2025-06-02 09:04:36 - physics_pipeline.src.downloaders.literature_downloader - INFO - Processing paper 237/256: On Characterising Assemblages in Einstein-Podolsky...
2025-06-02 09:04:36 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Searching for paper: On Characterising Assemblages in Einstein-Podolsky...
2025-06-02 09:04:38 - physics_pipeline.src

2025-06-02 09:05:06 - physics_pipeline.src.downloaders.literature_downloader - WARNING - ✗ Failed to download: A Single Quantum Cannot Be Cloned...
2025-06-02 09:05:06 - physics_pipeline.src.downloaders.literature_downloader - INFO - Processing paper 244/256: A Classical Interpretation of the Scrooge Distribu...
2025-06-02 09:05:06 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Searching for paper: A Classical Interpretation of the Scrooge Distribu...
2025-06-02 09:05:08 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Downloading paper: 1808.05179
2025-06-02 09:05:08 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - PDF downloaded: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/literature/1808.05179.pdf
2025-06-02 09:05:08 - physics_pipeline.src.utils.file_utils - INFO - Extracted main tex file: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/li

2025-06-02 09:05:34 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - Downloading paper: 1908.04305
2025-06-02 09:05:34 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - PDF downloaded: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/literature/1908.04305.pdf
2025-06-02 09:05:35 - physics_pipeline.src.utils.file_utils - INFO - Extracted main tex file: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/literature/1908.04305.tex
2025-06-02 09:05:35 - physics_pipeline.src.downloaders.arxiv_searcher - INFO - TEX downloaded and extracted: /Users/fanza/Desktop/Projects/AcademicAssistantExperiments/physics_synthesis_project_modular/documents/literature/1908.04305.tex
2025-06-02 09:05:36 - physics_pipeline.src.downloaders.literature_downloader - INFO - ✓ Successfully downloaded: 1908.04305
2025-06-02 09:05:36 - physics_pipeline.src.downloaders.literature_downloader - INF

In [ ]:
# =============================================================================
# CELL 4: Build Knowledge Base (WITH PYTORCH FIX)
# =============================================================================

# PYTORCH COMPATIBILITY FIX - Add this first!
import torch

if not hasattr(torch, 'get_default_device'):
    def get_default_device():
        """Fallback implementation for older PyTorch versions."""
        if torch.cuda.is_available():
            return torch.device('cuda')
        else:
            return torch.device('cpu')
    
    torch.get_default_device = get_default_device
    print("✅ Applied PyTorch compatibility fix")

# Now the imports should work
print("\n🏗️ BUILDING KNOWLEDGE BASE")
print("=" * 35)

from src.core import KnowledgeBase

# Initialize knowledge base
kb = KnowledgeBase(
    embedding_model=config.embedding_model,
    chunk_size=config.chunk_size,
    chunk_overlap=config.chunk_overlap
)

print("⏳ Building knowledge base from all documents...")
print("💡 This processes:")
print("   📚 Downloaded literature")
print("   📝 Your own papers") 
print("   ✏️  Current drafts")
print("   🧠 Creates semantic embeddings")
print()

# Build knowledge base
stats = kb.build_from_directories(
    literature_folder=config.literature_folder,
    your_work_folder=config.your_work_folder,
    current_drafts_folder=config.current_drafts_folder
)

print("✅ KNOWLEDGE BASE BUILT!")
print(f"   📚 Total documents: {stats.get('total_documents', 0)}")
print(f"   ✅ Successfully processed: {stats.get('successful_documents', 0)}")
print(f"   🧩 Total text chunks: {stats.get('total_chunks', 0)}")
print(f"   📊 Total words: {stats.get('total_words', 0):,}")

# Show source breakdown
source_breakdown = stats.get('source_breakdown', {})
if source_breakdown:
    print(f"\n📁 Source breakdown:")
    for source_type, info in source_breakdown.items():
        count = info.get('successful', 0) if isinstance(info, dict) else info
        print(f"   {source_type}: {count} documents")

# Save knowledge base
kb.save_to_file(config.cache_file)
print(f"\n💾 Knowledge base saved to: {config.cache_file}")

# Store for next cell
globals()['knowledge_base'] = kb

In [ ]:
# =============================================================================
# CELL 5: Test Literature-Aware Chat (IF YOU HAVE API KEY)
# =============================================================================

print("\n🤖 TESTING LITERATURE-AWARE CHAT")
print("=" * 40)

try:
    config.validate_api_keys()
    
    from src.chat import LiteratureAssistant
    
    # Create literature assistant
    assistant = LiteratureAssistant(
        knowledge_base=kb,
        anthropic_api_key=config.anthropic_api_key,
        chat_config=config.get_chat_config()
    )
    
    print("✅ Literature assistant created!")
    print(f"📊 Knowledge base: {kb.get_statistics().get('total_documents', 0)} documents")
    
    # Simple chat function
    def chat(question, show_sources=True):
        """Simple chat with the literature assistant."""
        print(f"\n🧑‍🔬 Question: {question}")
        print("🤖 Assistant: ", end="", flush=True)
        
        try:
            response = assistant.ask(question)
            print(response.content)
            
            if show_sources and response.sources_used:
                print(f"\n📚 Sources: {', '.join(response.sources_used)}")
            
        except Exception as e:
            print(f"Error: {e}")
        
        print("\n" + "-" * 50)
    
    print("\n💬 READY FOR CHAT!")
    print("💡 Try these commands:")
    print("   chat('What papers do I have in my knowledge base?')")
    print("   chat('What are the main research topics?')")
    print("   chat('Explain quantum entanglement')")
    
    # Store chat function for easy use
    globals()['chat'] = chat
    globals()['assistant'] = assistant
    
except ValueError as e:
    print("⚠️  Cannot test chat - API key not set")
    print("💡 Set your ANTHROPIC_API_KEY environment variable to test chat")

# =============================================================================
# CELL 6: Summary and Next Steps
# =============================================================================

print("\n🎉 PHYSICS PIPELINE SETUP COMPLETE!")
print("=" * 45)

# Final statistics
bib_count = len(list(config.biblio_folder.glob("*.bib")))
lit_count = len(list(config.literature_folder.iterdir())) if config.literature_folder.exists() else 0
work_count = len(list(config.your_work_folder.iterdir())) if config.your_work_folder.exists() else 0

print(f"📊 FINAL STATUS:")
print(f"   📚 .bib files: {bib_count}")
print(f"   📄 Downloaded papers: {lit_count}")
print(f"   📝 Your papers: {work_count}")
print(f"   🧠 Knowledge base: {'✅ Built' if 'knowledge_base' in globals() else '❌ Not built'}")
print(f"   🤖 Chat ready: {'✅ Yes' if 'chat' in globals() else '❌ Need API key'}")

print(f"\n🚀 WHAT YOU CAN DO NOW:")
if 'chat' in globals():
    print("   • chat('your question here') - Ask the AI assistant")
    print("   • assistant.synthesize_literature('topic') - Get research synthesis")
    print("   • assistant.help_with_writing('task') - Get writing help")
else:
    print("   • Add .bib files and run cells above")
    print("   • Set ANTHROPIC_API_KEY for chat functionality")

print(f"\n📁 KEY LOCATIONS:")
print(f"   📚 Add .bib files: {config.biblio_folder}")
print(f"   📝 Add your papers: {config.your_work_folder}")
print(f"   📄 Downloaded papers: {config.literature_folder}")
print(f"   💾 Knowledge base cache: {config.cache_file}")

print("\n🎯 Your physics literature synthesis pipeline is ready! 🔬📚")

In [ ]:
# Test 1: Overview of your collection
chat("What papers do I have in my knowledge base?")

In [ ]:
chat("Do you know what measurement-induced entanglement phase transitions are?")